# App to find Cheap Flights

# Introduction:

In 2014, the cheapest fare from New York to Vienna was found to be around $800, but according to the advertised fares, where for a select no. of dates, these tickets were between $350 and $450. 

It all seemed to be a good deal and one might wonder if whether it is true or not. The industry does mistake the occasional mistakes on fares, because airlines occasionally and accidentally do happen to post fares that exclude fuel surcharges. Normally, it is expected that the advanced algorithms employed by these airlines would be updating fares that takes into account large number of factors, however due to the order generations of systems in place, mistakes do happen.



# 1 Import the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from time import sleep

# 2 - Retrieving the Data from scraping the web:

Fare data are obtained from a AJAX-based (Asynchornous JavaScript) webpage, this will require a browser to do the work. For such a task, there will be a need for two of the following pacakges: Selenium and ChromeDriver.

- Selenium is a package for automating web browsers.
- ChromeDriver is a headless browser, meaning there isn't a user interface.

In [ ]:
from bs4 import BeautifulSoup
from selenium import webdriver

chromeDriver_file = 'chromedriver'
chromeDriver_file_conda = 'chromedriver-binary alias'

import os
path = os.path.abspath(chromeDriver_file)
print('pathway to ChromeDriver is: ' + '\n' + path)

# Set the ChromeDriver pathway:
chromeDriver_path = path

browser = webdriver.Chrome(chromeDriver_path)

### 2.1 Set the URL (from google flights):

Dates are set to 1st of June to 15th of June in the year 2020 (note: that these dates can be changed to anything).

NOTE: Need to use the Freebase IDs for city/region of interest for travel. for example, m/06y57 is for Sydney. m/0f04v is for empty search. m/02_286 is for NYC.

It is possible to find it when searching in google from th ebelow link:
https://www.google.com.mx/travel/guide?q=New+York+City&sa=X&rlz=1C1CHBD_esMX769MX769&output=search&tra=%5B%22AMAbHIJDZRALeKKuHEbLXHGOJ3aS9zzCTg:1579328461567%22,%22syndey%22,%22/m/02_286%22%5D&tcfs=EhUKCS9tLzAyXzI4NhIITmV3IFlvcms&dest_mid=/m/06y57#dest_mid=/m/06y57&tcfs=EiwKCC9tLzA2eTU3EgZTeWRuZXkaGAoKMjAyMC0wMi0wMxIKMjAyMC0wMi0wNw

And to confirm it with the link below: make sure to control+f and search for Freebase ID.
https://www.wikidata.org/wiki/Q3130

In [ ]:
# # Input webpage as string:
# flight_web_sats = 'https://www.google.com/travel/explore?tfs=CBsQAxojagcIARIDU1lEEgoyMDIwLTA2LTAxcgwIBBIIL20vMDJqOXoaI2oMCAQSCC9tLzAyajl6EgoyMDIwLTA2LTE1cgcIARIDU1lEcAFAAUgB&curr=AUD&gl=au&hl=en&authuser=0&origin=https%3A%2F%2Fwww.google.com&dest_mid'

# # Retrieve the webpage's content using Selenium:
# browser.get(flight_web_sats)

# # Check the title of the webpage:
# browser.title

In [ ]:
# # Check to see if the required information from the webpage was captured: Take a Screenshot and save as 'test_flights.png'.
# current_work_directory = os.getcwd()
# sleep(30)
# browser.save_screenshot(current_work_directory + '/test_flights.png')

### 2.2 Parsing the DOM to extract the individual flight data from the HTML tags:

Document Object Model (DOM) is the collection of the individual elements on a webpage. These will include things like HTML tags, like 'body' and 'div', or classes and IDs.

In [ ]:
# # Parsing:
# soup = BeautifulSoup(browser.page_source, "html.parser")

#### Extract the individual city data:

In [ ]:
# # Get the city data:
# # At the time of HTML scraping, the flight data was inside <div class='MeBuN'>
# # Or the by XPATH: //*[@id="flt-app"]/c-wiz/c-wiz/nav/div[1]/nav/div/div[2]/ol/li[1]/div

# sleep(10)

# flight_cards = soup.select('div[class*=MebuN]')

# # Check out a single flight card:
# flight_cards[0]


From the piece of HTML information above, it can be noticed that the information needed is within the markup. 
For example: 
- the destination is seen here 'class="W6bZuc YMlIz" London',
- where the duration of the flight is 'class="Xq1DAb" 1 day 3 hr 20 min' 
- and prices are located at 'class="QB2Jof xLPuCe" datags="CidHQ2lQekJHLS0tLS0tLS0tcGZiMTI5QUFBQUFGNGlrY29Bby1aQUESATAaCwj+wAQQAhoDVVNE">$739'.

#### Next, is to obtain the required data from the markup:


In [ ]:
# # For-loop to extract the relevant information:
# # At the time of HTML scraping, the price information are stored in <div class='MJg7fb'>

# # for card in flight_cards:
# #     print(card.select('h3')[0].text)
# #     print(card.select('div[class*=MJg7fb]')[0].text)
# #     print('\n')

# # Perform clean up of 'Great value' tags before the prices:
# for card in flight_cards:
#     print(card.select('h3')[0].text)
#     print(card.select('div[class*=MJg7fb]')[0].text.replace('Great value',""))
#     print('\n')

# browser.quit()

# print('Testing Complete.')

From the above, it can be confirmed that it is possible to retrieve the relevant data from the HTML. 

#### Next, is to retrieve flights that presents with the lowest cost and non-stop fares from the starting destination to the arrival destination. These flights would be for a 26 week period. All this is done by making a full scrape and parsing of a large number of fares.

#### Import the required libraries:

In [ ]:
import datetime
from datetime import date, timedelta
from time import sleep

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

In [ ]:
# Checks dict:
'''
Func: check if the structure is empty or not.
'''
def is_empty(any_structure):
    if any_structure:
#         print('Structure is not empty.')
        return False
    else:
#         print('Structure is empty.')
        return True

In [ ]:
#======= Restart ChromeDriver =========
path = os.path.abspath(chromeDriver_file)
print('pathway to ChromeDriver is: ' + '\n' + path)
print('\n')

# Set the ChromeDriver pathway:
chromeDriver_path = path

browser = webdriver.Chrome(chromeDriver_path)

#======= Scraping the Web Data =========

week_period = 26
start_date = '2020-06-01'
end_date = '2020-06-15'
# start_date = '2020-04-01'
# end_date = '2020-04-15'
Adjust_delay = 50 # Adjust for internet connection.
check_time = 7 # 7 days is teh default.\

check_wrong_startDate = datetime.datetime.strptime(start_date, '%Y-%m-%d')
check_wrong_date = check_wrong_startDate - timedelta(days = check_time)
check_wrong_date = str(check_wrong_date.date())

departure_destination = "Sydney"
arrival_destination = "Europe"



# Format the flight dates: with the python datetime standard.
startFlight_date = datetime.datetime.strptime(start_date, '%Y-%m-%d')
endFlight_date = datetime.datetime.strptime(end_date, '%Y-%m-%d')

# Dictionary for Fares:
flightFare_dict = {}

for idx in range(week_period):
    sat_start = str(startFlight_date).split()[0]
    sat_end = str(endFlight_date).split()[0]
    flightFare_dict.update({sat_start: {}})
    
    # Load webpage:
    sats = "https://www.google.com/flights?hl=en#flt=.." + sat_start + "*.." + sat_end + ";c:AUD;e:1;sd:1;t:h"
    sleep(np.random.randint(3,7))
    browser.get(sats)
    print('Index: ' + str(idx) + ' Starting Browser and searching link: Google ' + browser.title + '. Dates are: ' + sat_start + ' and ' + sat_end + '.' )
    
    # Input information to search for flights:
    wait_10sec = WebDriverWait(browser, 10) # Seconds of Waiting.

    print('Link Loaded, Entering Travel Details now.')

    # Departure Search: input of departure location.
    departureDestination_link = wait_10sec.until(EC.presence_of_element_located((By.XPATH, '//*[@id="flt-app"]/div[2]/main[1]/div[4]/div/div[3]/div/div[2]/div[1]')))
    departureDestination_link.click()
    departureDestination_link = wait_10sec.until(EC.presence_of_element_located((By.XPATH, '//*[@id="sb_ifc50"]/input')))
    sleep(1)
    departureDestination_link.send_keys(departure_destination)
    sleep(2)
    departureDestination_link.send_keys(Keys.ENTER)

    # Arrival Search: input of arrival location.
    arrivalDestination = wait_10sec.until(EC.presence_of_element_located((By.XPATH, '//*[@id="flt-app"]/div[2]/main[1]/div[4]/div/div[3]/div/div[2]/div[2]')))
    arrivalDestination.click()
    arrivalDestination = wait_10sec.until(EC.presence_of_element_located((By.XPATH, '//*[@id="sb_ifc50"]/input')))
    sleep(1)
    arrivalDestination.send_keys(arrival_destination)
    sleep(2)
    arrivalDestination.send_keys(Keys.ENTER)

    # Get new URL:
    sleep(1)
    new_browser_url = browser.current_url
    print('After inputting the destinations and searching, the new URL is: \n' + new_browser_url)

    # Finally, click on the 'Search' button:
    floatingActionButton_click = browser.find_elements_by_xpath('//*[@id="flt-app"]/div[2]/main[1]/div[4]/div/div[3]/div/div[4]/floating-action-button')[0]
    sleep(2)
    floatingActionButton_click.click()
    print('Search done. Next is to get a list of the travel information.')
    
    
    # Extract Relevant Data from webpage:
    print('Collecting data.')
    sleep(Adjust_delay)
    soup = BeautifulSoup(browser.page_source, 'html.parser')
    flight_cards = soup.select('div[class*=MebuN]')
    
    previous_sat_start_date = 0
    
    for card in flight_cards:
        print('Extracting...')
        while True:
            try:
                city = card.select('h3')[0].text
                fare = card.select('div[class*=MJg7fb]')[0].text.replace('Great value',"")    
                print(city)
                print(fare)
                print('\n')
                flightFare_dict[sat_start] = {**flightFare_dict[sat_start], **{city: fare}}
                   
            except RuntimeError as detail:
                print('Handling run-time error: ' + detail)
                continue
            break  


    # Checks if the DICT in this current loop is empty, if YES, redo the data extraction:
    previous_sat_start_date = startFlight_date - timedelta(days = check_time)
    previous_sat_start_date = str(previous_sat_start_date.date())
    
    if ( previous_sat_start_date == sat_start ):
        continue   
    
    elif ( previous_sat_start_date == check_wrong_date):
        continue
    
    elif ( is_empty(flightFare_dict[previous_sat_start_date]) ):
        print('This current index loop DICT was found to be empty, Retrying to collect Data.')
        sleep(1)
        for card in flight_cards:
            print('Alternate try of extracting...')
            city = card.select('h3')[0].text
            fare = card.select('div[class*=MJg7fb]')[0].text.replace('Great value',"")    
            print(city)
            print(fare)
            print('\n')
            flightFare_dict[sat_start] = {**flightFare_dict[sat_start], **{city: fare}}
    else:
        print('Error in collecting information, RETRY has FAILED.')
        break
    
    
    # Update the date: Add 7 days.
    startFlight_date = startFlight_date + timedelta(days = check_time)
    endFlight_date = endFlight_date + timedelta(days = check_time)
    print('\n')
    
browser.quit()
print('Quiting Broswer, Data Collection Complete.')

In [ ]:
# is_empty(flightFare_dict[previous_sat_start_date]) 

In [ ]:
# startFlight_date

In [ ]:
flightFare_dict

In [ ]:
# flight_cards = soup.select('div[class*=MebuN]')
# flight_cards

## 2.3 Check out the data collected from the 26 week period:

Begin with checking out flights heading to Berlin.

In [ ]:
# Checking out outward flights to Berlin:

city_key = 'Berlin'
for key in flightFare_dict:
    print(key, flightFare_dict[key][city_key])

It can be seen that there will be some need of further cleaning of the data, such as removing the 'A$'for AUD dollars from the price data and to remove the ',' commas.

In [ ]:
city_dict = {}

for k, v in flightFare_dict.items():
    city_dict.update({k: int(v[city_key].replace(',','').split('$')[1])})

In [ ]:
city_dict

After some cleaning, it can be seen that the values appears as wanted. 

#### Plot the data:

In [ ]:
# Convert prices to integers:
prices = [int(x) for x in city_dict.values()]

# Extract the Date data from the dictionary:
dates = city_dict.keys()

# Plot:
fig, ax = plt.subplots(figsize = (10, 6))
plt.scatter(x = dates,
            y = prices,
            color = 'black',
            s = 50           
           )
ax.set_xticklabels(dates, rotation = -70)

From the plot above, where they are flights from Sydney to Berlin, shows 26 consecutive weeks of data and have some variations to these fares. These can range from 900 AUD dollars to 1350 AUD dollars. It can also be said that initially from just eyeballing it, there isn't a certain pattern to these fares.

#### Now, having a look at another City: Milan.

In [ ]:
# Checking out outward flights to Milan:

city_key = 'Milan'
for key in flightFare_dict:
    print(key, flightFare_dict[key][city_key])

In [ ]:
city_dict = {}

for k, v in flightFare_dict.items():
    city_dict.update({k: int(v[city_key].replace(',','').split('$')[1])})

In [ ]:
# Convert prices to integers:
prices = [int(x) for x in city_dict.values()]

# Extract the Date data from the dictionary:
dates = city_dict.keys()

# Plot:
fig, ax = plt.subplots(figsize = (10, 6))
plt.scatter(x = dates,
            y = prices,
            color = 'black',
            s = 50           
           )
ax.set_xticklabels(dates, rotation = -70)

From the plot above, where they are flights from Sydney to Milan and the fares ranges from 1150 AUD dollars to 2250 AUD dollars. The types of cheap fares of interests can be seen from the dates of 26th Oct to 16th of Nov (around 1150 AUD dollars). These are the Outliers of interest.

#### Next is to create an outlier detection system to notify the user of such cheap fares

# 3 Outlier Detection System: for cheap flights:

Outliers can be describe as an observation that lies outside the overall pattern of a distribution. (ref: http://mathworld.wolfram.com/Outlier.html)
Such outliers can be determined by techniques that are both parametric and non-parametric. Once of the techniques utilised is the density-based spatial clustering of application with noise (DBSCAN). Another is called Grubb's Test. Depending on the type of data that present itself, where it can be multivariate or univariate, the technique utilised is different.

For the purpose of this notebook, the data is a univariate time-series data, and that the technique to be used is called the Generalised Extreme Studentised Deviate (GESD) test specifically for outliers. (ref: http://www.real-statistics.com/students-t-distribution/identifying-outliers-using-t-distribution/generalized-extreme-studentized-deviate-test/)


## 3.1 Import the required Libraries:

In [ ]:
from scipy import stats

#### Plot the data with probability plot:

In [ ]:
fix, ax = plt.subplots(figsize = (10, 6))
stats.probplot(list(city_dict.values()), plot = plt)
plt.show()

From the plot above, . This assesses the normal probability (or the Q-Q plot, quantile-quantile). The diagonal straight line in the plot is the 'normal' line, and that any data close to this straight line is considered normal and not an outlier. The outliers are data points that veers off away from the straight line, or presents with a strong S-shape. 

In the case where there are more data available, it is more likely that the pattern/trend will be approximate to the diagonal straight line.

## 3.2 Outlier detection code:

#### Import the required library:

In [ ]:
from PyAstronomy import pyasl

In [ ]:
fare_prices = prices
max_outliers = 3
significance_lvl = 0.025 #lower value means less sensitive to false positives in the algorithm

r = pyasl.generalizedESD(fare_prices, max_outliers, significance_lvl, fullOutput = True)
print('City -> ' + city_key)
print('Total Outliers: ', r[0])

# Print out data in regards to 'R' and Lambda: used to determine if data point is an outlier.
out_dates = {}
for i in sorted(r[1]):
    out_dates.update({list(dates)[i]: list(prices)[i]})   
print('Outlier Dates', out_dates.keys(), '\n')
print('        R         Lambda')

# Make plot: outliers are in red colour.
for i in range(len(r[2])):
    print('%2d %8.5f %8.5f' % ((i+1), r[2][i], r[3][i]))
    
fig, ax = plt.subplots(figsize = (10, 6))
plt.scatter(dates, prices, color = 'black', s=50)
ax.set_xticklabels(dates, rotation =-70);

for i in range(r[0]):
    plt.plot(r[1][i], prices[r[1][i], 'rp'])